In [1]:
using static System.Console;
using System.Text.Json;
using System.Text.Json.Serialization;

public interface IDiscriminatedUnion<T1,T2> where T1: class where T2: class
{
    string DataType {get;}
    void Match(Action<T1> type1, Action<T2> type2);
    TResult Match<TResult>(Func<T1, TResult> type1, Func<T2, TResult> type2);
}

public class  DiscriminatedUnion<T1,T2>: IDiscriminatedUnion<T1, T2> where T1: class where T2: class
{
    public string DataType => this.GetType().Name;

    public static Type GetSubType(string name)
    {
        var typeofT1 = typeof(T1);
        if(typeofT1.Name.Equals(name, StringComparison.OrdinalIgnoreCase))
        {
            return typeofT1;
        }

        var typeofT2 = typeof(T2);
        if(typeofT2.Name.Equals(name, StringComparison.OrdinalIgnoreCase))
        {
            return typeofT2;
        }

         throw new ArgumentException($"Unknow type: {name}");
    }

    public void Match(Action<T1> type1, Action<T2> type2)
    {
        (this switch
        {
            T1 t1 => (Action)(() => type1(t1)),
            T2 t2 => () => type2(t2),
            _ => () => throw new Exception("Unknown data type")
        })();
    }

    public TResult Match<TResult>(Func<T1, TResult> type1, Func<T2, TResult> type2)
    {
        return this switch{
            T1 t1 => type1(t1),
            T2 t2 => type2(t2),
            _ => throw new Exception("Unknown data type")
        };
    }
}

public abstract class  Animal:DiscriminatedUnion<Animal.Dog, Animal.Cat>
{
    public class Dog: Animal
    {
        public string DogName {get; set;}

        public void Bark()
        {
            WriteLine("Bark");
        }
    }

    public class Cat: Animal
    {
        public string CatName {get; set;}

        public void Meow()
        {
            WriteLine("Meow");
        }
    }
}

public class AnimalTypeConverter: JsonConverter<Animal>
{
    public override bool CanConvert(Type objectType) => typeof(Animal).IsAssignableFrom(objectType);

    public override Animal Read(ref Utf8JsonReader reader, Type typeToConvert, JsonSerializerOptions options)
    {
        // Check for null values
        if (reader.TokenType == JsonTokenType.Null) return null;
        
        // Copy the current state from reader (it's a struct)
        var readerAtStart = reader;

        // Read the `dataType` from our JSON document
        using var jsonDocument = JsonDocument.ParseValue(ref reader);
        var jsonObject = jsonDocument.RootElement;

        var dataType = jsonObject.GetProperty("DataType").GetString();
        
        // See if that class can be deserialized or not

        if (!string.IsNullOrEmpty(dataType))
        {
            // Deserialize it
            return JsonSerializer.Deserialize(ref readerAtStart, Animal.GetSubType(dataType)) as Animal;
        }
        
        throw new NotSupportedException($"{dataType ?? "<unknown>"} can not be deserialized");
    }

    public override void Write(Utf8JsonWriter writer, Animal value, JsonSerializerOptions options)
    {
        JsonSerializer.Serialize(writer, value, value.GetType());
    }
}

Animal animal = new Animal.Dog { DogName = "Dog"};
animal = new Animal.Cat { CatName= "Cat"};

animal.Match(
(dog) => WriteLine($"{dog.DogName}, tag: {dog.DataType}"),
(cat) => WriteLine($"{cat.CatName}, tag: {cat.DataType}"));

var serializerOptions = new JsonSerializerOptions();
serializerOptions.Converters.Add(new AnimalTypeConverter());
WriteLine(JsonSerializer.Serialize(animal, animal.GetType(), serializerOptions));

var deserialized = JsonSerializer.Deserialize<Animal>("{\"CatName\":\"Cat\",\"DataType\":\"Cat\"}", serializerOptions);
deserialized.Match(
(dog) => WriteLine($"{dog.DogName}, tag: {dog.DataType}"),
(cat) => WriteLine($"{cat.CatName}, tag: {cat.DataType}"));

Cat, tag: Cat


{"CatName":"Cat","DataType":"Cat"}


Cat, tag: Cat
